In [1]:
import numpy as np
import pandas as pd
import requests
import config
import os
import json
from pandas.io.json import json_normalize

##Schedule_a API guide: https://api.open.fec.gov/developers/#/receipts/get_schedules_schedule_a_

In [83]:
### MULTI-CANDIDATE SEARCH ###
#Set search for all 2020 (two-year transaction period) Maine Congressional candidates
cycle='2020'
p=1
parameters = {'cycle':cycle
            ,'api_key':config.api_key
            ,'committee_type':'P'
            ,'per_page':'100'
            ,'page':p}

#Requests candidate info
r = requests.get('https://api.open.fec.gov/v1/committees/',params=parameters).json()

#Loop through pages
while p <= r['pagination']['pages']:

    print('page: '+str(p)+','+str(r['pagination']['per_page'])+' records')

    #Pull results
    r = requests.get('https://api.open.fec.gov/v1/committees',params=parameters).json()

    #Add results to dataframe
    df = json_normalize(r['results'])
    dfs.append(df)
    p=p+1
    parameters.update(page=p)

print('Job Complete')

cands=pd.concat(dfs,ignore_index=True,sort=False)
cands=cands.drop_duplicates(subset='committee_id')
cands=cands[(cands.committee_type_full=='Presidential') 
            & (cands['affiliated_committee_name'].str.lower() != 'none') ]
cands.reset_index(drop=True)

#Extract committee IDs
comm_ids=cands['committee_id'].tolist()

page: 1,100 records
page: 2,100 records
page: 3,100 records
page: 4,100 records
page: 5,100 records
page: 6,100 records
page: 7,100 records
page: 8,100 records
Job Complete


,affiliated_committee_name,candidate_ids,committee_id,committee_type,committee_type_full,cycles,designation,designation_full,filing_frequency,first_file_date,...,federal_funds_flag,has_raised_funds,inactive_election_years,incumbent_challenge,incumbent_challenge_full,last_f2_date,load_date,office,office_full,principal_committees
0,DAVID PARTY AFFILIATION,[],C00692772,P,Presidential,"[2018, 2020]",P,Principal campaign committee,Q,2018-12-13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,None,[P60020963],C00674580,P,Presidential,"[2018, 2020]",P,Principal campaign committee,Q,2018-03-22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,None,[P60021243],C00686048,P,Presidential,"[2018, 2020]",P,Principal campaign committee,Q,2018-08-23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AMERICA FIRST,[P00013102],C00715276,P,Presidential,[2020],P,Principal campaign committee,Q,2019-08-09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,DAVID WIATR,"[H8NJ01226, P00006544]",C00661397,P,Presidential,"[2018, 2020]",U,Unauthorized,Q,2017-11-23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,IMMORTAL TECHNIQUE,[P00007096],C00671792,P,Presidential,"[2018, 2020]",P,Principal campaign committee,Q,2018-03-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,None,[P40003584],C00564880,P,Presidential,"[2014, 2016, 2018, 2020]",P,Principal campaign committee,Q,2014-06-17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,DEMOCRATED PARTY PACS,[P80003916],C00709600,P,Presidential,[2020],P,Principal campaign committee,Q,2019-06-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,None,[P60004983],C00653527,P,Presidential,"[2018, 2020]",P,Principal campaign committee,Q,2017-08-21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,None,[P00007898],C00682096,P,Presidential,"[2018, 2020]",P,Principal campaign committee,Q,2018-07-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
#Initialize dataframe collector for itemized contribs
dfs=[]
id=int(0)

#Loop through pages
for x in range(len(comm_ids)):
    
# print('FOR')
    #Initialize query dict
    querydict = {'per_page':'100'
                  ,'committee_id':comm_ids[id]
                  ,'sort':'contribution_receipt_date'
                  ,'api_key':config.api_key
                  ,'is_individual':'true'
                  ,'two_year_transaction_period':cycle
                 }

    r = requests.get('https://api.open.fec.gov/v1/schedules/schedule_a/',params=querydict).json()
    
    #Results rows
    df = json_normalize(r['results'])
    dfs.append(df)
    
    #Last page variables
    if r['pagination']['last_indexes'] is None:
        None
        if id<len(comm_ids[0]):
                id=id+1
        else:
            break
    else: 
        last_index=pd.to_numeric(r['pagination']['last_indexes']['last_index'])
        last_date=r['pagination']['last_indexes']['last_contribution_receipt_date']
        
        #Loop through pages
        while id<len(comm_ids[0]):

#print('WHILE')

            #Initialize query, without last_index and date
            querydict.update([('last_index',last_index)
                            ,('last_contribution_receipt_date',last_date)])

            r = requests.get('https://api.open.fec.gov/v1/schedules/schedule_a/',params=querydict).json()

            #Results rows
            df = json_normalize(r['results'])
            dfs.append(df)

            #Update Pagination variables
            last_index=pd.to_numeric(r['pagination']['last_indexes']['last_index'])
            last_date=r['pagination']['last_indexes']['last_contribution_receipt_date']        
            dfs.append(df)
            
            #Increment to max ID in list
            if id<len(comm_ids[0]):
                id=id+1
            else:
                break

#Concatenate all dfs
itemdf=pd.concat(dfs,sort=False)
itemdf=itemdf.drop_duplicates(subset='transaction_id')

TypeError: 'NoneType' object is not subscriptable

In [93]:
#Initialize dataframe collector for itemized contribs
dfs=[]
id=int(0)

#Loop through pages
for x in range(len(comm_ids)):
    
    print('FOR')
    #Initialize query dict
    querydict = {'per_page':'100'
                  ,'committee_id':comm_ids[id]
                  ,'sort':'contribution_receipt_date'
                  ,'api_key':config.api_key
                  ,'is_individual':'true'
                  ,'two_year_transaction_period':period
                 }

    r = requests.get('https://api.open.fec.gov/v1/schedules/schedule_a/',params=querydict).json()
    
    #Results rows
    df = json_normalize(r['results'])
    dfs.append(df)
    
    #Last page variables
    if r['pagination']['last_indexes'] is None:
        None
        if id<len(comm_ids[0]):
                id=id+1
        else:
            break
    else: 
        last_index=pd.to_numeric(r['pagination']['last_indexes']['last_index'])
        last_date=r['pagination']['last_indexes']['last_contribution_receipt_date']
        
        #Loop through pages
        while id<len(comm_ids[0]):

            print('WHILE')

            #Initialize query, without last_index and date
            querydict.update([('last_index',last_index)
                            ,('last_contribution_receipt_date',last_date)])

            r = requests.get('https://api.open.fec.gov/v1/schedules/schedule_a/',params=querydict).json()

            #Results rows
            df = json_normalize(r['results'])
            dfs.append(df)

            #Update Pagination variables
            last_index=pd.to_numeric(r['pagination']['last_indexes']['last_index'])
            last_date=r['pagination']['last_indexes']['last_contribution_receipt_date']        
            dfs.append(df)
            
            #Increment to max ID in list
            if id<len(comm_ids[0]):
                id=id+1
            else:
                break

#Concatenate all dfs
itemdf=pd.concat(dfs,sort=False)
itemdf=itemdf.drop_duplicates(subset='transaction_id')

FOR
FOR
FOR
FOR
FOR
FOR
FOR
FOR
WHILE


TypeError: 'NoneType' object is not subscriptable

In [94]:
r['pagination']['last_indexes']['last_index']

TypeError: 'NoneType' object is not subscriptable

In [6]:
json_normalize(data=r_cands['results'],record_path='principal_committees')

,affiliated_committee_name,candidate_ids,committee_id,committee_type,committee_type_full,cycles,designation,designation_full,filing_frequency,first_file_date,last_f1_date,last_file_date,name,organization_type,organization_type_full,party,party_full,state,treasurer_name
0,NONE,[P00011569],C00631770,P,Presidential,"[2018, 2020]",P,Principal campaign committee,Q,2017-01-25,2017-01-25,2018-04-17,753 2020,None,None,DEM,DEMOCRATIC PARTY,IL,"SEVENFIVETHREE, JO MR."
1,NONE,[P00006270],C00653931,P,Presidential,"[2018, 2020]",P,Principal campaign committee,Q,2017-08-25,2017-08-25,2017-08-25,THE MATTHEW M. ABRAUGH FOR PRESIDENT,None,None,NON,NON PARTISAN,OH,"ABRAUGH, MATTHEW M. MR."
2,NONE,[P00009217],C00693770,P,Presidential,[2020],P,Principal campaign committee,Q,2019-01-12,2019-01-12,2019-01-12,BRANDON ACKER FOR AMERICA,None,None,DEM,DEMOCRATIC PARTY,NC,"WEBBER, MARK"
3,NONE,[P00009563],C00694885,P,Presidential,[2020],P,Principal campaign committee,Q,2019-01-28,2019-01-28,2019-01-28,CAROLINE AEBI,None,None,DEM,DEMOCRATIC PARTY,CA,"AEBI, ESTHER CAROLINE"
4,NONE,[P00009852],C00695650,P,Presidential,[2020],P,Principal campaign committee,Q,2019-02-02,2019-02-02,2019-02-02,CARLOS ALCANTARA FOR PRESIDENT,None,None,NNE,NONE,FL,"ALCANTARA, CARLOS"
5,NONE,[P00012252],C00708438,P,Presidential,[2020],P,Principal campaign committee,Q,2019-06-05,2019-06-05,2019-06-05,COMMITTEE TO ELECT M.LATROY ALEXANDRIA-WILLIAM...,None,None,DEM,DEMOCRATIC PARTY,TN,"THREATT, NELL MRS WILLIAMS"
6,NONE,[P00010975],C00700625,P,Presidential,[2020],P,Principal campaign committee,Q,2019-03-30,2019-03-30,2019-03-30,FRANZ ALIQUO: THE PEOPLE'$ VOICE,None,None,DEM,DEMOCRATIC PARTY,NY,"ALIQUO, FRANZ RPMGRP/ART"
7,NONE,[P00008854],C00694646,P,Presidential,[2020],P,Principal campaign committee,Q,2019-01-23,2019-01-31,2019-01-31,DARCIE ALLEN FOR PRESIDENT 2020,None,None,IND,INDEPENDENT,SC,"ALLEN, DARCIE J"
8,NONE,[P00005561],C00631390,P,Presidential,"[2018, 2020]",P,Principal campaign committee,Q,2017-01-20,2017-01-20,2017-01-20,THE PRESIDENTIAL CAMPAIGN COMMITTEE FOR ANTHON...,None,None,NNE,NONE,MN,"ALONGI, ANTHONY"
9,NONE,[P00007906],C00697318,P,Presidential,[2020],P,Principal campaign committee,Q,2019-02-21,2019-02-21,2019-04-08,DONNA JEAN ALSTON,None,None,DEM,DEMOCRATIC PARTY,DC,"ALSTON, DONNA JEAN MISS"


0                                   PAC - Nonqualified
1                                                House
2                                               Senate
3                                                House
4                                               Senate
5                                                House
6                                         Presidential
7                                                House
8                                         Presidential
9                                         Presidential
10                                        Presidential
11                                              Senate
12                                               House
13                                               House
14                                              Senate
15                                              Senate
16                                              Senate
17                                              Senate
18        

In [84]:
#Write itemized individual results to local CSV
cwd = os.getcwd()
cands.to_csv(cwd+'/data/presidential-committees.csv')

HttpError: <HttpError 400 when requesting https://sheets.googleapis.com/v4/spreadsheets/12bHf1qEtKtGGje0a3lBJJ8r-IvYcs3yl5jxbP6jNMCo/values/Raw%20Itemized%20filings%21A8926%3AAC10711?valueInputOption=USER_ENTERED&alt=json returned "Range ('Raw Itemized filings'!A8926:AC10711) exceeds grid limits. Max rows: 8925, max columns: 84">